<a href="https://colab.research.google.com/github/cassiomatheus/Intelig-ncia-Computacional/blob/main/Adaboost_Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Adaboost

# Questão **1**

R = Para a questão indicada, devemos considerar os pesos iniciais normalizados de $ \mathbf{W_{t}} = [0.1, 0.4, 0.2, 0.1, 0.2]$, onde sua decision stump com menor erro ponderado $\mathrm{\epsilon_{t}}$ foi   $\\\mathrm{I} (\mathrm{x_{1}}\leqslant1.5)$, calculando os valores $\mathrm{\epsilon_{t}}$, $\mathrm{\beta_{t}}$, $\mathrm{\alpha_{t}}$ e  $\mathrm{e_{i}}$ (para t=0, tivemos $w_2$ com erro):


.$\tag{1} \epsilon_t = min_{f,p,\theta} \sum_{i=1}^n w_i |h(x_i, f, p, \theta)-y_i| $

\begin{align}
\epsilon_t = 0,2\ (w_2=0,2)
\end{align}
.$$\tag{2} \beta_t = \frac{\epsilon_t}{1-\epsilon_t} =  \frac{0,2}{1-0,2} = 0,25 $$
.
$$\tag{3} \alpha_t = ln(\frac{1}{\beta_t})= 1,38 $$
.
$$\tag{4} e_0 =0 , e_1 =0 , e_2 =1,e_3 =0,e_4 =0$$ 


# Questão **2**

A) O classificador forte possui três stumps determinadas pelo número de interações e seus erros ponderados encontradas em cada stump. ()

.$\tag{1} C(x) = 1 \ \mathrm{para} \sum_{t=1}^T \alpha_t \cdot h_t (x) \geq  \frac{1}{2} \sum_{t=1}^T \alpha_t $
.
\begin{align}
C(x) = 1.94*0.5+2.56*0.5+2.03*(0.16+0.16+0.16) = 3,27
\end{align}

B) para o classificador forte temos que lembrar a sua taxa de aprendizado (laerning_rate=1), os pesos encontrados para t = 3 e outros parâmetros descritos abaixo:

.$\tag{1} \epsilon_t = min_{f,p,\theta} \sum_{i=1}^n w_i |h(x_i, f, p, \theta)-y_i| $
.
\begin{align}
\epsilon_t = 0,038 +0,038+  0,038=0,11
\end{align}
.$$\tag{2} \beta_t = \frac{\epsilon_t}{1-\epsilon_t} =  \frac{0,11}{1-0,11} = 0,13 $$
.
$$\tag{3} \alpha_t = ln(\frac{1}{\beta_t})= 2,04 $$
.
$$\tag{4} e_0 =0 , e_1 =0 , e_2 =1,e_3 =0,e_4 =0$$ 


C) A stump com maior peso na decisão foi a da terceira interação, onde se obteve maior erro, propondo um peso maior nestas features, considerando que o algoritmo Adaboost necessita dos erros ponderados (obs: atentar-se aos outliers, podendo causar ruídos em seu desempenho)

$$W_{3,4} ,W_{3,5}, W_{3,6}.$$ 

D) O valor predito foi 1 para o vetor.

E) A escolha foi fornecida através do peso = 8 considaderado na feature $x_0$, o qual temos uma chance de acerto maior com uma zona de decisão próxima a ela, levando em conta $x_0=-2000;y=1$.

F) Calculando as variáveis t=0:

$\tag{1} \epsilon_t = min_{f,p,\theta} \sum_{i=1}^n w_i |h(x_i, f, p, \theta)-y_i| $
.
\begin{align}
\epsilon_t = w_3 = 0.06
\end{align}
.$$\tag{2} \beta_t = \frac{\epsilon_t}{1-\epsilon_t} =  \frac{0,06}{1-0,06} = 0,064 $$
.
$$\tag{3} \alpha_t = ln(\frac{1}{\beta_t})= 2,75 $$
.
$$\tag{4} e_0 =0 , e_1 =0 , e_2 =0,e_3 =1,e_4 =0,e_5 =0,e_6 =0,e_7 =0$$ 

Cod.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import export_text
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier
import numpy as np

T = 3 #number of weak learners
show_training_info = True

print(__doc__)

def load_lista1_dataset():
    #from numpy import genfromtxt
    my_data = np.genfromtxt('/content/drive/MyDrive/4 Semestre/Inteligência Computacional/lista1_dataset.csv', delimiter=',')
    X = my_data[:,:2] # two first parameters are input vector
    #y = my_data[:,2:]
    y = np.ravel(my_data[:,2:],order='C') #convert column vector into 1D array
    return X,y

def load_simple():
    #from numpy import genfromtxt
    my_data = np.genfromtxt('/content/drive/MyDrive/4 Semestre/Inteligência Computacional/dataset_validation.txt', delimiter=',')
    X = my_data[:,:2] # fish length and weight
    y = np.ravel(my_data[:,2:],order='C') #convert column vector into 1D array
    return X,y

def adaboost_predict(x, weak_classifiers, alphas, num_weak_classifiers):
    decision_threshold = 0.5 * np.sum(alphas)
    score = 0
    for t in range(num_weak_classifiers):
        score += alphas[t] * weak_classifiers[t].predict(x)
    return np.array([score >= decision_threshold]).astype(int)

def weighted_error(prediction, correct_label, weights):
    error_indices = np.argwhere(prediction != correct_label)
    return np.sum(weights[error_indices])

if False:
    X, y = load_lista1_dataset()
else:
    X, y = load_simple()
print('O valor')
print(X)
num_training_examples = X.shape[0]

#Initialization of instance weights
sample_weight = np.array([8, 1, 1, 1, 1, 1, 1, 2]) #zeros(y.shape, dtype=float) 
index_of_positive_classes = np.argwhere(y==1)
index_of_negative_classes = np.argwhere(y==0)
num_positive = len(index_of_positive_classes)
num_negative = len(index_of_negative_classes)
print('Initial sample_weight', sample_weight)

# sample_weight[index_of_positive_classes] = 1.0 / (2*num_positive)
# sample_weight[index_of_negative_classes] = 1.0 / (2*num_negative)

print('Initial sample_weight', sample_weight)

weak_classifiers = list()
alphas = list()
for t in range(T):
    print("######## Iteration t=",t)
    #1) normalize weight
    weights_sum = np.sum(sample_weight)
    sample_weight = sample_weight / weights_sum

    if show_training_info:
        print('sample_weight=',sample_weight)

    #2) select the best weak classifier with respect to weighthed error
    decision_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1)
    decision_stump.fit(X, y, sample_weight)

    #3) Define the weak classifier h_t(x)
    weak_classifiers.append(decision_stump)

    #4) Update the weights based on the decision of the classifier that
    #incorporates all previously designed weak classifiers
    y_pred = decision_stump.predict(X)

    epsilon_weighted_error = weighted_error(y_pred, y, sample_weight)
    #AK: function below return strange, large values
    #epsilon_weighted_error = decision_stump.score(X, y, sample_weight)

    beta = epsilon_weighted_error / (1.0 - epsilon_weighted_error)
    this_alpha = np.log(1.0/beta)
    alphas.append(this_alpha)

    predictions_of_weak_classifier_t = decision_stump.predict(X)

    errors_i = np.array((predictions_of_weak_classifier_t != y)).astype(int)
    sample_weight *= beta ** (1.0-errors_i)
    #sample_weight = sample_weight / np.sum(sample_weight)

    if show_training_info:
        print('predictions_of_weak_classifier_t=',predictions_of_weak_classifier_t)
        print('                  correct labels=',y)
        print('epsilon_weighted_error=', epsilon_weighted_error)
        #print('sample_weight=',sample_weight)
        print('beta_t=',beta,'alpha_t=',this_alpha)
        print('errors_i=',errors_i)

for t in range(T):
    print('t=', t, ', alpha_t=',alphas[t])
    r = export_text(weak_classifiers[t]) #, feature_names=feature_names) #, class_names=target_names)
    print(r)
print('threshold =', 0.5 * np.sum(alphas))

predictions_of_strong_classifier = adaboost_predict(X, weak_classifiers, alphas, T)
print('predictions_of_strong_classifier=',predictions_of_strong_classifier)
print('                          labels=',y)

x = np.genfromtxt('/content/drive/MyDrive/4 Semestre/Inteligência Computacional/simple - Copia.txt', delimiter=',')
x1 = x[:,:2] # two first parameters are input vector
#y = my_data[:,2:]
y1 = np.ravel(x[:,2:],order='C') #convert column vector into 1D array
predictions_of_strong_classifier = adaboost_predict(x1, weak_classifiers, alphas, T)
print('predictions_of_strong_classifier=',predictions_of_strong_classifier)
print('                          labels=',y1)

print('\n\n#### Comparison with scikit-learn ####')
ada_discrete = AdaBoostClassifier(
    base_estimator=decision_stump,
    learning_rate=1,
    n_estimators=T,
    algorithm="SAMME")
ada_discrete.fit(X, y)
ada_discrete_pred = ada_discrete.predict(X)
error_indices = np.argwhere(ada_discrete_pred != y)
print('error_indices=',error_indices)
print('Scikit-learn misclassification error rate (%)=',100.0*len(error_indices)/num_training_examples)

print('Scikit-learn AdaBoost classifier:')
weak_learners_list = ada_discrete.estimators_
for t in range(len(weak_learners_list)):
    print('t=',t,'alpha?=',ada_discrete.estimator_weights_[t])
    r = export_text(weak_learners_list[t]) #, feature_names=feature_names) #, class_names=target_names)
    print(r)




Automatically created module for IPython interactive environment
O valor
[[ 2.e+03 -2.e+00]
 [ 4.e+03 -1.e+00]
 [ 3.e+03 -1.e+00]
 [-2.e+03  1.e+00]
 [-6.e+03 -1.e+00]
 [-4.e+03  0.e+00]
 [-6.e+03 -4.e+00]
 [-2.e+03  3.e+00]]
Initial sample_weight [8 1 1 1 1 1 1 2]
Initial sample_weight [8 1 1 1 1 1 1 2]
######## Iteration t= 0
sample_weight= [0.5    0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.125 ]
predictions_of_weak_classifier_t= [0. 0. 0. 1. 1. 1. 1. 1.]
                  correct labels= [0. 0. 0. 0. 1. 1. 1. 1.]
epsilon_weighted_error= 0.0625
beta_t= 0.06666666666666667 alpha_t= 2.70805020110221
errors_i= [0 0 0 1 0 0 0 0]
######## Iteration t= 1
sample_weight= [0.26666667 0.03333333 0.03333333 0.5        0.03333333 0.03333333
 0.03333333 0.06666667]
predictions_of_weak_classifier_t= [0. 0. 0. 0. 1. 1. 1. 0.]
                  correct labels= [0. 0. 0. 0. 1. 1. 1. 1.]
epsilon_weighted_error= 0.06666666666666667
beta_t= 0.07142857142857142 alpha_t= 2.6390573296152584
errors_i= [0 0